WNixalo | 20181108-9

---

This gets rank 255/938 on the kaggle planet amazon competition, @ **0.92367** pvt w/ a threshold of `0.25`.

## start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai import *
from fastai.vision import *

In [3]:
path = Config.data_path()/'planet'
path.mkdir(exist_ok=True)
path

PosixPath('/home/jupyter/.fastai/data/planet')

## re-init all if need

In [4]:
# ! conda install -c haasad eidl7zip -y

In [ ]:
# ! 7za -bd -y -so x {path}/train-jpg.tar.7z | tar xf - -C {path}

In [5]:
!ls {path}

In [20]:
# ! pip install kaggle --upgrade
# ! mkdir -p ~/.kaggle/
# ! mv ~/kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json

In [63]:
# # download
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train-jpg.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f train_v2.csv -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg.tar.7z -p {path}
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test-jpg-additional.tar.7z -p {path}  
# ! kaggle competitions download -c planet-understanding-the-amazon-from-space -f test_v2_file_mapping.csv -p {path}
! kaggle competitions download -c planet-understanding-the-amazon-from-space -f sample_submission_v2.csv -p {path}

  0%|                                                | 0.00/154k [00:00<?, ?B/s]
100%|████████████████████████████████████████| 154k/154k [00:00<00:00, 67.3MB/s]


In [7]:
!ls {path}

test-jpg-additional.tar.7z  test_v2_file_mapping.csv.zip  train_v2.csv.zip
test-jpg.tar.7z		    train-jpg.tar.7z


In [64]:
# decompress & clean
# ! unzip {path}/train_v2.csv.zip -d {path}
# ! unzip {path}/test_v2_file_mapping.csv.zip -d {path}
! unzip {path}/sample_submission_v2.csv.zip -d {path}
# ! 7za -bd -y x {path}/train-jpg.tar.7z -o{path}
# ! 7za -bd -y x {path}/test-jpg.tar.7z -o{path}
# ! 7za -bd -y x {path}/test-jpg-additional.tar.7z -o{path}
# ! tar -xf {path}/test-jpg-additional.tar -C {path}
# ! tar -xf {path}/train-jpg.tar -C {path}
# ! tar -xf {path}/test-jpg.tar -C {path}

Archive:  /home/jupyter/.fastai/data/planet/sample_submission_v2.csv.zip
  inflating: /home/jupyter/.fastai/data/planet/sample_submission_v2.csv  
  inflating: /home/jupyter/.fastai/data/planet/__MACOSX/._sample_submission_v2.csv  


In [65]:
! rm -rf {path}/*.zip
# ! rm -rf {path}/*.7z
# ! rm -rf {path}/*.tar

## Data

In [14]:
df = pd.read_csv(path/'train_v2.csv')

In [15]:
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

In [8]:
np.random.seed(0)

In [30]:
src = (ImageFileList.from_folder(path)
       .label_from_csv('train_v2.csv', sep=' ', folder='train-jpg', suffix='.jpg')
       .random_split_by_pct(0.0001))

In [31]:
data = (src.datasets(ImageMultiDataset)
        .transform(tfms, size=128)
        .databunch().normalize(imagenet_stats))

In [32]:
data.valid_ds.ds

ImageMultiDataset of len 4

## MultiClass

I want to see how far I can get with a ResNet34 before going to 50.

In [33]:
arch = models.resnet34

a threshold of 0.25 got me best results last time, so I'll see if putting that will help get more accurate metrics.

In [34]:
f_score = partial(fbeta, thresh=0.25)
learn = create_cnn(data, arch, metrics=[accuracy_thresh, f_score])

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/jupyter/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:01<00:00, 55463352.89it/s]


### Stage 1 size 128

In [35]:
lr = 0.01

In [36]:
learn.fit_one_cycle(5, slice(lr))

Total time: 05:08
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.118840    0.079255    0.985294         0.972470  (01:02)
2      0.111048    0.062136    0.985294         0.946023  (01:01)
3      0.102709    0.059693    0.970588         0.984375  (01:01)
4      0.098300    0.045486    0.985294         0.968750  (01:01)
5      0.091363    0.045031    0.985294         0.984375  (01:01)



In [37]:
learn.save('stage-1-rn34-all')

In [38]:
learn.fit_one_cycle(5, slice(lr))

Total time: 05:07
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.099128    0.056947    1.000000         0.934118  (01:01)
2      0.100135    0.028738    1.000000         1.000000  (01:01)
3      0.100602    0.040140    1.000000         1.000000  (01:01)
4      0.093511    0.031510    1.000000         1.000000  (01:01)
5      0.089607    0.028908    1.000000         1.000000  (01:01)



In [39]:
learn.save('stage-1-rn34-all')

### Stage 2 size 128

In [40]:
learn.unfreeze()

In [41]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

Total time: 07:28
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.096939    0.035154    1.000000         1.000000  (01:30)
2      0.095430    0.042647    1.000000         0.984375  (01:29)
3      0.091046    0.028918    1.000000         1.000000  (01:29)
4      0.088895    0.038921    1.000000         1.000000  (01:29)
5      0.084451    0.033910    1.000000         1.000000  (01:29)



In [42]:
learn.save('stage-2-rn34-all')

In [43]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

Total time: 07:28
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.086097    0.038472    1.000000         0.988095  (01:29)
2      0.092633    0.033085    1.000000         0.984375  (01:29)
3      0.087016    0.030257    1.000000         1.000000  (01:29)
4      0.083042    0.035571    1.000000         1.000000  (01:29)
5      0.080665    0.029056    1.000000         1.000000  (01:29)



In [44]:
learn.save('stage-2-rn34-all')

### Stage 1 size 256

In [45]:
data = (src.datasets(ImageMultiDataset)
        .transform(tfms, size=256)
        .databunch().normalize(imagenet_stats))

learn.data = data
data.train_ds[0][0].shape

torch.Size([3, 256, 256])

In [49]:
learn.freeze()

In [50]:
lr = 1e-2/2

In [51]:
learn.fit_one_cycle(5, slice(lr))

Total time: 16:38
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.088853    0.052580    1.000000         0.972470  (03:20)
2      0.086033    0.053445    1.000000         0.943058  (03:19)
3      0.086782    0.029156    1.000000         1.000000  (03:19)
4      0.085321    0.035531    1.000000         0.972470  (03:19)
5      0.079460    0.036163    1.000000         0.972470  (03:19)



In [52]:
learn.save('stage-1-256-rn34-all')

In [53]:
learn.fit_one_cycle(5, slice(lr))

Total time: 16:36
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.082377    0.039151    1.000000         0.972470  (03:19)
2      0.083979    0.033664    1.000000         1.000000  (03:19)
3      0.084648    0.045617    1.000000         0.972470  (03:19)
4      0.082936    0.038716    1.000000         0.972470  (03:18)
5      0.082616    0.034059    1.000000         0.972470  (03:19)



In [54]:
learn.save('stage-1-256-rn34-all')

### Stage 2 size 256

In [55]:
learn.unfreeze()

In [56]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

Total time: 23:02
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.084451    0.039155    1.000000         0.988095  (04:36)
2      0.085343    0.054677    0.985294         0.958683  (04:36)
3      0.084148    0.052638    0.985294         0.949743  (04:36)
4      0.081343    0.032835    0.985294         0.988095  (04:36)
5      0.076463    0.032336    1.000000         0.988095  (04:36)



In [57]:
learn.save('stage-2-256-rn34-all')

In [58]:
learn.fit_one_cycle(5, slice(1e-5, lr/5))

Total time: 23:01
epoch  train_loss  valid_loss  accuracy_thresh  fbeta   
1      0.079911    0.045833    0.985294         0.972470  (04:36)
2      0.079621    0.039752    1.000000         0.972470  (04:36)
3      0.079370    0.031724    0.985294         0.988095  (04:36)
4      0.074065    0.037323    0.985294         0.988095  (04:36)
5      0.073343    0.039684    0.985294         0.988095  (04:36)



In [59]:
learn.save('stage-2-256-rn34-all')

## Test set

In [200]:
# learn.load('stage-2-256-rn34-all')

In [60]:
idx2class = idx2class = {v:k for k,v in learn.data.train_ds.ds.class2idx.items()}

In [62]:
!ls {path}

__MACOSX  test-jpg	       test_v2_file_mapping.csv  train_v2.csv
models	  test-jpg-additional  train-jpg


In [66]:
subm_df = pd.read_csv(path/'sample_submission_v2.csv')
subdic = {c1:c2 for c1,c2 in zip(subm_df.image_name,subm_df.tags)}

In [67]:
def set_test_datafolder(learner, test_folder):
    learner.data = (src.add_test_folder(test_folder)
                    .datasets(ImageMultiDataset)
                    .transform(tfms, size=256)
                    .databunch().normalize(imagenet_stats))

1st dataset portion

In [68]:
set_test_datafolder(learn, 'test-jpg')
raw_preds = learn.TTA(ds_type=DatasetType.Test)

Total time: 12:03



In [69]:
len(raw_preds), raw_preds[0].shape, raw_preds[1].shape

(2, torch.Size([40669, 17]), torch.Size([40669, 17]))

In [70]:
preds = copy(raw_preds)

In [71]:
fnames = [f.name.split('.')[0] for f in learn.data.test_ds.ds.x]

2nd dataset portion

In [72]:
set_test_datafolder(learn, 'test-jpg-additional')
raw_preds = learn.TTA(ds_type=DatasetType.Test)

Total time: 06:14



In [73]:
preds = (torch.cat((preds[0],raw_preds[0])), torch.cat((preds[1],raw_preds[1])))

In [141]:
# # NOTE: this will work:
# preds = [torch.cat((preds[i],raw_preds[i])) for i in range(len(preds))]

# # but this will leave you with a <generator object <genexpr> at 0x7f91696821a8>
# preds = (torch.cat((preds[i],raw_preds[i])) for i in range(2))

In [74]:
fnames.extend([f.name.split('.')[0] for f in learn.data.test_ds.ds.x])

## Submitting predictions to Kaggle

In [75]:
import datetime
date = str(datetime.date.today()).replace('-','')

In [92]:
th = 0.2775

for i,fn in enumerate(fnames):
    subdic[fn] = ' '.join(idx2class[cdx] for cdx in np.where(preds[0][i]>=th)[0])
    
subm_col = [subdic[fn] for fn in subm_df.image_name]
subm_df.tags = subm_col

In [84]:
# !mkdir {path}/submissions

In [93]:
sub_name = f'planet_stage-2-256-rn34-all_{date}_th{str(th)}.csv'
subm_df.to_csv(path/'submissions/'/sub_name, index=False, )
! kaggle competitions submit -c planet-understanding-the-amazon-from-space -f {path}/submissions/{sub_name} -m "fastai 1.0; 2-stg; RN34; threshold="{th}

100%|██████████████████████████████████████| 2.11M/2.11M [00:01<00:00, 1.27MB/s]
Successfully submitted to Planet: Understanding the Amazon from Space

'stage-2-256-rn34_1'|threshold|pvt score|rank|'stage-2-256-rn34'|pvt score|rank
-|-|-|-|-|-|-|-|-
#|0.5|0.89458|-|#|0.89430|-
#|0.4|0.91274|-|#|-|-
#|0.3|0.92177|-|#|0.92046|-
#|0.2|0.91901|-|#|-|-
#|0.25|0.92250|-|#|-|-
#|0.2575|0.92255|-|#|0.92201|-

Training on the entire dataset with a threshold of 0.25 gets **0.92367** pvt @ 255/983.

I may go to resnet50.

## end